In [2]:
import torch
import torch.nn as nn

In [32]:
import numpy as np
a = np.nan
if not np.isnan(a):
    print("yes")
else:
    print("no")

no


In [100]:
import numpy as np
import torch
import cv2

def spatial_avg(patch,coordinates,kernel_size=3):
    (X,Y,c) = coordinates
    data = patch[:,:,c].copy()
    while np.isnan(data[X,Y]):
        max_x = data.shape[0]-1
        max_y = data.shape[1]-1
        start_x = max(0,X-kernel_size//2)
        start_y = max(0,Y-kernel_size//2)
        end_x = min(max_x,X+kernel_size//2)+1
        end_y = min(max_y,Y+kernel_size//2)+1
        print("coordinates: ",coordinates," start_x start_y end_x end_y max_x max_y: ",start_x ,start_y, end_x ,end_y, max_x ,max_y)
        neighbours = []
        for i in range(start_x,end_x):
            for j in range(start_y,end_y):
                neighbours.append(data[i,j])
        # Check if all elements are nan
        all_nan = all(np.isnan(x) for x in neighbours)
        if all_nan:
            kernel_size+=2
            continue
        data[X,Y] = np.nanmean(neighbours)
        patch[:,:,c] = data
    return patch
                
def fill_nan(patch,coordinates): # coordinats = (x,y,c)
    data = patch.copy()
    (x,y,c) = coordinates
    if (c==0):
        if not np.isnan(data[x,y,c+1]):
            data[x,y,c] = data[x,y,c+1]
        else:
            data = spatial_avg(data,(x,y,c))
    elif (c==255):
        if not np.isnan(data[x,y,c-1]):
            data[x,y,c] = data[x,y,c-1]
        else:
            data = spatial_avg(data,(x,y,c))
    else:
        if not np.isnan(data[x,y,c+1]):
            data[x,y,c] = data[x,y,c+1]
        elif not np.isnan(data[x,y,c-1]):
            data[x,y,c] = data[x,y,c-1]
        else:
            data = spatial_avg(data,(x,y,c))
    return data
import torch.nn.functional as F
# Handling NaN Values
patch = np.load(r"D:\HSI Project\Updated_Work\HSI_Classification\Minerals_Dataset\Patches_Data\Patches\130_A_143173.npy")
# print(patch.shape)
print(np.isnan(patch).any())
for k in range(10):
    print(patch[:,:,k])
ind = np.where(np.isnan(patch))
print(ind)
# print(ind)
# for k in range(10):
#     print(patch[:,:,k])
for (X,Y,c) in zip(ind[0],ind[1],ind[2]):
    patch = fill_nan(patch,(X,Y,c)) 
    
# print("2")
# print(patch.shape)

# patch = np.transpose(F.pad(torch.tensor(np.transpose(patch,axes=(2,0,1))),(1,1,1,1)).numpy(),axes=(1,2,0))
# patch = torch.tensor(patch).permute(2,0,1).unsqueeze(0)


# print(patch.shape)

for k in range(10):
    print(patch[:,:,k])
print(np.isnan(patch).any())

True
[[    nan     nan 0.0405  0.04758 0.04153]
 [    nan     nan 0.04044     nan     nan]
 [    nan     nan 0.0341      nan     nan]
 [0.04074     nan     nan 0.031   0.02795]
 [    nan     nan     nan     nan     nan]]
[[    nan     nan 0.01486     nan     nan]
 [0.03848 0.02917 0.02559 0.02219 0.02306]
 [0.03262 0.02345     nan     nan     nan]
 [0.02916 0.02423     nan     nan     nan]
 [0.04095 0.03757 0.03177     nan     nan]]
[[    nan     nan 0.02966 0.0343  0.03018]
 [    nan     nan 0.03412 0.03192 0.02101]
 [0.05127 0.03796 0.02618 0.02979 0.02806]
 [0.03345 0.02426 0.03348 0.03796 0.03117]
 [0.04074 0.0339  0.02408 0.02133 0.03506]]
[[    nan     nan     nan     nan     nan]
 [    nan     nan     nan     nan     nan]
 [    nan     nan     nan     nan     nan]
 [    nan     nan     nan     nan 0.02122]
 [0.03424 0.0247  0.02707 0.02849 0.02898]]
[[0.04135 0.03763 0.0419  0.046   0.04495]
 [0.04208 0.03217 0.03308 0.03534 0.02547]
 [0.03656 0.03516 0.03043 0.02827 0.0284 ]
 [

In [22]:
class HSI_3DCNN(nn.Module):
    def __init__(self, in_chs, patch_size,class_nums):
        super(HSI_3DCNN,self).__init__()
        self.in_chs = in_chs
        self.patch_size = patch_size
        self.conv3dmodel = nn.Sequential(
                            nn.Conv3d(in_channels=1,out_channels=16,kernel_size=(24, 3, 3)),
                            nn.BatchNorm3d(16),
                            nn.ReLU(inplace=True),
                            nn.Conv3d(in_channels=16,out_channels=32,kernel_size=(12, 3, 3)),
                            nn.BatchNorm3d(32),
                            nn.ReLU(inplace=True),
                            nn.Dropout(0.3),
                            nn.MaxPool3d(kernel_size=(8,2,2))
                            )
        temp = self.conv3dmodel(torch.rand(2,1,in_chs,patch_size,patch_size))
        self.dim3d = temp.view(2,-1).shape[1]
        self.linear = nn.Sequential(
            nn.Linear(self.dim3d,200),
            nn.BatchNorm1d(200),
            nn.ReLU(inplace=True),
            nn.Linear(200,class_nums)
        )

    def forward(self,X):
        x = self.conv3dmodel(X)
        x = x.view(x.shape[0],-1)
        x = self.linear(x)
        return x
    


In [10]:
32*27

864

In [ ]:
# class HSI_3DCNN(nn.Module):
#     def __init__(self,num_classes):
#         super(HSI_3DCNN, self).__init__()
#         self.model = nn.Sequential(
#         nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(24, 5, 5)),
#         nn.BatchNorm3d(32),
#         nn.ReLU(inplace=True),
#         nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(16, 5, 5)),
#         nn.BatchNorm3d(64),
#         nn.ReLU(inplace=True),
#         nn.MaxPool3d(kernel_size=(1, 2, 2))
#         )
#         x = torch.rand(1,1,50,11,11)
#         num_channels = self.model(x).view(1,-1).shape[1]
#         self.fc1 = nn.Linear(num_channels, 300)
#         self.bn3 = nn.BatchNorm1d(300)
#         self.fc2 = nn.Linear(300, num_classes)

#     def forward(self, x):
#         x = self.model(x)
#         x = x.view(x.size(0), -1)  # Flatten the output
#         x = self.fc1(x)
#         x = self.bn3(x)
#         x = torch.relu(x)
#         x = self.fc2(x)
#         return x

In [23]:
x = torch.rand(2,1,256,7,7)
m = HSI_3DCNN(256,7,76)
m(x).shape

torch.Size([2, 76])